**Le fichier *nom_prenom_OPT_M1*.ipynb est à rendre avant vendredi 7 décembre à 18h (heure de New York)**

# TP1 : Optimisation sans contraintes and beyond

Dans la suite nous considérons $\Omega=(0,1)$ et regarderons le problème de minimisation suivant :
\begin{equation}
\label{P}
\min_{u\in\mathcal{K}}\mathcal{J}[u],
\tag{$\mathcal{P}$}
\end{equation}
où $$ \mathcal{J}[u]:=\int_\Omega (\frac{1}{2}u'(x)^2-f(x)u(x))dx $$
et
$$ \mathcal K:=\{ u\in H^1(\Omega)\;|\;u(0)=a,u(1)=b\; \text{donnés}\}. $$

On rappelle que $H^1(\Omega):=\{u\;|\; u\in L^2,\;u'\in L^2 \}$ où $u'$ est la derivée au sens des distributions et $L^2$ désigne l'espace de Lebesgue des fonctions de carré intégrable.

Dans ce TP on veut se focaliser sur les méthodes de **gradient** (pas fixe, pas optimal, pas d'Armijo) et la méthode du **gradient conjugué**.

**Good to know : éléments finis de Lagrange, cas $P_1$**

Un cadre adpaté pour approcher la solution de ce problème est offert par la méthode des éléments finis de Lagrange :
soit $0=x_0<x_1<\cdots<x_{N+1}=1$ un maillage de $\Omega$.Définissons alors l’espace $\mathcal V_h$, sous-espace de $\mathcal K$ de dimension finie, par
$$ \mathcal V_h:=\{ v_h\in\mathcal C^0(\Omega)\;|\; v_h(x)=v_h(x_i)+(x-x_i)\frac{v_h(x_{i+1})-v_h(x_{i})}{h_i}\;\forall x\in[x_i,x_{i+1}]\;\text{et}\; v_h(0)=a,v_h(1)=b\}.$$
On approche  la solution $u$ du problème par $u_h\in\mathcal V_h$. En paticulier, comme $\mathcal V_h=span\{\phi_1,\cdots,\phi_N\}$ où $\phi_i(x_j)=\delta_{ij}$, on peut decomposer $u_h$ sous la forme $u_h(x)=\sum_{i=1}^{N}u_i\phi_i(x)$.

Le problème approché devient alors
\begin{equation}
\label{Ph}
\min_{u_h\in\mathcal V_h} J[u_h]
\tag{$\mathcal{P}_h$}
\end{equation}

### Partie I : Équation de Laplace

1- Expliciter le problème \eqref{Ph} et montrer qu'il est équivalent, si on approche les intégrale avec la formule de trapèzes, au suivant
$$ \min_{u_N\in\mathbb{R}^N} \mathcal J_N[u_N]$$
où 
\begin{equation}
\label{PN}
\mathcal J_N[V]=\sum_{i=0}^N\Big(\dfrac{(V_{i+1}-V_i)^2}{2h_i}-\frac{h_i}{2}(f_{i+1}V_{i+1}+f_iV_i)\Big),
\tag{$\mathcal P_N$}
\end{equation}
$f_i=f(x_i)$, $h_i=x_{i+1}-x_i$ et $V_{N+1}=b, V_0=a$.

2- Montrer que \eqref{PN} peut se mettre sous la forme 
$$ A_N\bf{u}_N=\bf{f}_N $$
où $A_N$ et $\bf{f}_N$ sont à preciser.

3- Y-a-t'il une relation entre la solution du problème \eqref{PN} et celle du problème ci-dessous discretisé pas les éléments finis P1 Lagrange sur le même mailage avec la même formule de quadrature?
\begin{equation}
\label{Lap}
\begin{cases}
&-u''(x)=f(x),\;x\in\Omega\\
& u(0)=a,\;u(1)=b.
\end{cases}
\tag{$\mathcal L$}
\end{equation}

4- On considère $f(x)=-1$, $a=0$, $b=\frac{1}{2}$ de sorte que la solution exacte de \eqref{Lap} est $u(x)=\frac{x^2}{2}$.
Résoudre le système linéaire trouvé au point (2) en utilisant `numpy.linalg.solve`, pour différentes valeurs de N et vérifier que la solution de \eqref{PN} converge bien pour la norme $H^1(\Omega)$ vers la solution de \eqref{P} lorsque $h\rightarrow 0$. (Pour chaque $N$ on prend un maillage uniforme $h=\frac{1}{N+1}$). Résoudre directement \eqref{PN} en utilisant `scipy.optimize.minimize`.

**DANS LA SUITE ON PRENDRA $f=1$, $a=b=0$**

5- Implementer $\mathcal J_N$ et $\nabla\mathcal J_N$ en utilsant les fonctions `J(U,a,b,x,f)` et `DJ(U,a,b,x,f)`.
On rappelle que pour `a,b,x,f` donnés, on peut définir une fonction dans la seule variable `V` comme `JN =lambda V : J(V,a,b,x,f)`.


### Partie II : Méthode du gradient à pas fixe

Soient  $\bf u_0$ un point de départ,  $t$ le pas de descente et  $\epsilon$ une tolerance alors 
 l'algorithme du gradient à pas fixe pour une fonctionnelle $\mathcal J_N :\mathbb R^N\rightarrow \mathbb R$ s'écrit comme
\begin{equation}
\label{grad}
\begin{cases}
&\bf{d}_k= -\nabla\mathcal J_N(\bf u_k),\\
&\bf u_{k+1}=\bf u_k+t\bf d_{k}.
\end{cases}
\end{equation}
On arrêtera les itérations lorsque $\parallel \nabla \mathcal J_N(\bf u_k)\parallel\leq \epsilon$.

On note d'abord que la fonctionnelle peut être écrite sous la forme $\mathcal J_N[\bf u]=\frac{1}{2}<\bf u, A\bf u>-<\bf b, \bf u>$ où $A\in\mathcal{M}_N(\mathbb{R})$ est une matrice symétrique et définie positive et $<\cdot,\cdot>$ désigne le produit scalaire euclidien.

**Etude théorique : convergence linéaire**
- On rappelle que si $A\in\mathcal{M}_n(\mathbb{R})$ est symétrique et $\lambda_1,\cdots,\lambda_n\in\sigma(A)$ alors on a $\parallel Ax\parallel\leq \max_i|\lambda_i|\parallel x\parallel$ pour tout $x\in\mathbb{R}^n$. Montrer que pour $t\in(0,\dfrac{2}{\lambda_{max}})$ on a l'inegalité suivante
$$ \parallel x_{k+1}-x_{k}\parallel\leq \max(|1-t\lambda_{min}|,|1-t\lambda_{max}|)\parallel x_{k}-x_{k-1}\parallel, $$
où $\lambda_{min}$ (resp. $\lambda_{max}$) est la plus petite (resp. grande) valeur propre de A.
En deduire que pour rendre le taux de convergence le plus petit possible, le meilleur choix est $t=\dfrac{2}{\lambda_{min}+\lambda_{max}}$. Calculer le taux et le commenter dans le cas où $\lambda_{min}<<\lambda_{max}$. 

6- Implémenter l'algorithme du gradient à pas fixe à travers une fonction de la forme 

`gradient_fixe(J,DJ,u0,t,epsilon,iterMax,store) `

prennant en argument la fonctionnelle `J` à minimiser, le gradient `DJ`, la valeur initiale `u0`, le pas `t`, le nombre maximal d'iterations autorisées `IterMax`, la tolerance `epsilon` et un parametre `store` pilotant le type de stockage dans `u` (0 ou 1). Cette fonction devra retourner:
- `u` dernier terme de la suite $\bf u_k$ si `store=0` ou tous les termes si `store=1`;

- `iter` nombre d'itérations effectuées.

- `err` liste de $\parallel \nabla \mathcal J_N(\bf u_k)\parallel$.


7- Tracer sur une même figure les courbes de niveaux de $\mathcal J_2$ (**N=2**) ainsi que le champ de vecteurs $\nabla\mathcal{J}_2$ sur $[-10,10]^2$. (Utiliser les fonctions `matplotlib.pyplot.contour` et `matplotlib.pyplot.quiver`).

8- Calculer les itérations $\bf u_k$ données par l'algorithme et tracer sur la même figure que précédemment la ligne qui relie les $\bf u_k$. On prendra $\bf u_0=(8,4)$, $t=0.1$ et $\epsilon=10^{-8}$. 

9- Pour $N=2,5,20,50,100,200$. Afficher à l'aide de la fonction `fprintf` le nombre d'itérations ainsi que le temps de calcul pour chaque $N$. Tracer sur la même figure les solutions approchées ainsi que la solution exacte de \eqref{Lap}. On prendra  $t=0.1$ et $\epsilon=10^{-8}$. Tracer en échelle logarithmique `err` en fonction du nombre des itérations. Commenter les résultats.

10- Reprendre l'expérience précédent en utlisant $t\in(0,\dfrac{2}{\lambda_{max}})$ et puis le meilleur choix de $t$ trouvé au début de l'exercice. Commenter les résultats. 

### Partie III : Méthode du gradient à pas variable
La suite $\bf u_k$  des itérées de l'algorithme de descente de gradient à pas variable est donnée par

\begin{equation}
\begin{cases}
&\bf{d}_k= -\nabla\mathcal J_N(\bf u_k),\\
&\bf u_{k+1}=\bf u_k+t_k\bf d_{k},
\end{cases}
\end{equation}

où $t_k$ pourra être le pas optimal ou le pas calculé en utilsant la règle d’Armijo vue en cours.
On arrêtera les itérations lorsque $\parallel \nabla \mathcal J_N(\bf u_k)\parallel\leq \epsilon$.

11- Sachant que la fonctionnelle est quadratique, calculer analytiquement le pas optimal $t_k$.

12- Pour determiner le pas $t_k$ fournir les trois approches suivantes :
- `optimal_t_analytique(J,DJ,uk,dk)` fonction prennant en argument la fonctionnelle, son gradient, l'itérée $\bf u_k$ et la direction de descente $\bf d_k$. La fonction doit retourner le pas optimal analytique tenant compte du caractère quadratique de $\mathcal J_N$;
- `optimal_t_newton(J,DJ,uk,dk,t0,tol)` fonction qui devra retourner le pas optimal en utilisant la méthode de Newton-Raphson. On rappelle la méthode de Newton-Raphson permet de trouver un point en lequel une fonction $f$ (on est en dimension 1) s'annule, connaissant une approximation $x_0$ de ce point. L'algorithme est donné par :
\begin{equation}
x_{k+1}=x_k-\dfrac{f(x_k)}{f'(x_k)},
\end{equation}
on arrêtera les itérations lorsque $|f(x_k)|<\text{tol}$
- `t_armijo(J,DJ,uk,dk,alpha,beta)` fonction qui devra retourner le pas en utilisant la règle d'Armijo (`t` et `beta` sont les paramètres à utiliser pour la règle d'Armijo). On rappelle la règle d'Armijo : soient $\alpha\in(0,\dfrac{1}{2})$ et $\beta\in(0,1)$ fixés, le pas $t$ est donné par
$$ t=\max\{ t\;|\; t=\beta^l\; l\in\mathbb{N},\; f(x+td)\leq f(x)+\alpha t <\nabla f(x),d>\}. $$

13- Programmer une fonction `gradient_variable(J,DJ,u0,t0,epsilon,iterMax,store, optim1D)` où `optim1D` sera un entier identifiant la méthode à utiliser pour calculer le pas optimal. La fonction devra retourner `u` dernier terme de la suite $\bf u_k$ si `store=0` ou tous les termes si `store=1`, `iter` nombre d'itérations effectuées, `tL` liste des pas optimaux et `err` liste de $\parallel \nabla \mathcal J_N(\bf u_k)\parallel$.

14- Cas $N=2$. Reprendre les expériences effectuées dans la méthode du gradient à pas fixe.

15- Reprendre la question (9). Pour chaque valeur de $N$, trancer un graphique en échelle logaritmique de `err` obtenu en utilisant le pas optimal et la régle d'Armijo. Commenter les résultats.

16- Modifier `gradient_variable` de sorte qu'elle retourne aussi les directions de descente $\bf d_k$ que l'on rangera dans les colonnes d'une matrice $D$.

17- Pour le **pas optimal analytique** et $N=30$, calculer $D^TD$, commenter le résultat et le montrer.



### Partie IV : Méthode du gradient conjugué

On rappelle que la méthode du gradient conjugué n'est valable que pour des fonctionnelles de la forme $\mathcal J_N[\bf u]=\frac{1}{2}<\bf u, A\bf u>-<\bf b, \bf u>$ où $A\in\mathcal{M}_N(\mathbb{R})$ est une matrice symétrique et définie positive et $<\cdot,\cdot>$ désigne le produit scalaire euclidien. L'algorithme est alors donné par

\begin{equation}
\begin{cases}
&t_k=-\dfrac{<\bf g_k,\bf d_k>}{<\bf d_k,A\bf d_k>},\\
&\bf u_{k+1}=\bf u_k+t_k\bf d_{k},\\
&\bf g_{k+1}=A \bf u_{k+1}-\bf b,\\
&\bf \beta_{k}=\dfrac{<\bf g_{k+1},A\bf d_{k}>}{<\bf d_{k},A\bf d_{k}>},\\
&\bf{d}_{k+1}= -\bf g_{k+1}+\beta_k\bf d_k,
\end{cases}
\end{equation}

18- Montrer que :
- $<\bf d_k,A\bf d_i>=0$ $\forall i<k$;
- $t_k=\dfrac{\parallel\bf g_k\parallel^2}{<\bf d_k,A\bf d_k>}$;
- $\bf g_{k+1}=\bf g_k+t_k A\bf d_k$;
- $<\bf g_{k+1},\bf g_{k}>=0$; 
- $\beta_{k}=\dfrac{\parallel\bf g_{k+1}\parallel^2}{\parallel\bf g_{k}\parallel^2}$

19- Programmer la fonction `gradient_conjugue(J,DJ, u0, epsilon, iterMax, store)` qui retournera `u` dernier terme de la suite $\bf u_k$ si `store=0` ou tous les termes si `store=1`, `iter` nombre d'itérations effectuées, `tL` liste des pas generés par l'agorithme et `err` liste de $\parallel \nabla \mathcal J_N(\bf u_k)\parallel$. On utilisera comme test d'arrêt $\parallel \bf g_k\parallel <\epsilon$.

20- Cas $N=2$. Reprendre les expériences effectuées dans la méthode du gradient à pas fixe.

21- Reprendre la question (9).


### Partie V : Méthode de Newton

On veut maintenant implementer la méthode de Newton :

\begin{equation}
\label{N}
\begin{cases}
&\bf {d}_{k}= - \nabla^2\mathcal{J}_N(\bf u_k)^{-1}\nabla \mathcal J_N(\bf u_k),\\
&\bf u_{k+1}=\bf u_k+\bf d_{k}.\\
\end{cases}
\end{equation}


22- Programmer la fonction `newton(J,DJ,D2J, u0, epsilon, iterMax, store)` qui retournera `u` dernier terme de la suite $\bf u_k$ si `store=0` ou tous les termes si `store=1` et `iter` nombre d'itérations effectuées. On utilisera comme test d'arrêt $\parallel g_k\parallel <\epsilon$. **(`D2J` est la hessienne de $\mathcal J_N$ )** Commenter les résultats obtenus (surtout le nombe d'itérations effectuées) en utilisant les $N$ de la question (9) (prendre $\bf u_0=\bf 0$).

